# Условие

Посчитайте для нескольких классов (небинарный) обощённый ROC-AUC:
$$ROC-AUC=\frac{\sum_{i, j: t_i>t_j} \left([y_i>y_j]+\frac{1}{2}[y_i=y_j]\right)}{\left|i,j:t_i>t_j\right|}$$
где $0\leq t_i,y_i\leq 10$ - вещественные числа, правильный ответ и предсказание модели.

# Решение

В процессе решения нам придётся сортировать оба списка, но мы всё ещё хотим сохранить связь между правильным ответом и его предсказанием, поэтому в программе каждый $y_i$ будет хранить ссылку на соответсвующий $t_i$ и еще на его индекс после сортировки. Но здесь я буду обозначать индекс соответсвующего элемента как $q(i)$, тогда соответсвующий $y_i$ элемент будет $t_{q(i)}$.

Отсортируем список $t$. Пройдёмся по нему и посчитаем количество объектов, больше которых $t_i$:
$$bigger_0=0$$
$$bigger_i=\begin{cases}
i, & t_i>t_{i-1} \\
bigger_{i-1}, & t_i=t_{i-1}
\end{cases}$$
Тогда:
$$\left|i,j:.t_i>t_j\right|=\sum_{i=0}^{N-1}bigger_i$$
Теперь пройдёмся в обратном порядке по списку $t$ и каждому объекту в соответствие поставим индекс ближайшего следующего элемента, который строго больше текущего:
$$next\ bigger_{N-1}=N$$
\- фиктивное значение, т.к. больше максимального элемента не существует,
$$next\ bigger_i=\begin{cases}
i+1, & t_i<t_{i+1} \\
next\ bigger_{i+1}, & t_i=t_{i-1}
\end{cases}$$
Теперь сортируем список $y=(y_i,t_{q(i)})$ по значениям $y_i$; если значения $y_i=y_j$, сортируем по соответсвующим им $t_{q(i)},t_{q(j)}$. Будем идти по списку в обратном порядке.
<br>
Посмотрим на самый большой элемент $y_{N-1}$. Он больше или равен всех значений $y$. Тогда соответсвующий ему $t_{q(N-1)}$ будет выше всех значений, выше которых он в списке $t$, т.е. относительно них его порядок не изменился:
|Сортированный $t$|Сортированный $y=(y_i,t_{q(i)})$|
|---|---|
|1|2 2|
|2|7 1|
|3|8 4|
|4|9 5|
|5|10 3|

Мы видим, что $t_i=3$ в своём списке выше 2 элементов, и в списке $y$ тоже. Значит его вклад в числитель совпадает со вкладом в знаменатель, порядок $t_i=3$ не изменился относительно порядка $t_j=1,t_k=2$. И мы его полный вклад $bigger_{q(N-1)}$ добавим в числитель.
<br>
Однако все элементы, которые больше по значению $t_i=3$ в списке $t$, свой вклад теряют на единицу, так как каждый теперь не выше $t_3$. Эту единицу вычтем из начального вклада каждого такого элемента $bigger_j$: с $j=next\ bigger_{q(N-1)}$ по $j=N-1$.
<br>
Аналогично будем делать для каждого следующего $y_i$.


**Замечание.** Вычитание единицы из интервала будет занимать много времени $O(N^2)$. Поэтому будем это делать с помощью *дерева отрезков* за $O(NlogN)$.


Что же делать с $y_i=y_j$? Взглянем на пример такой ситуации:
||10|9|8|7|6|5|4|3|
|---|---|---|---|---|---|---|---|---|
|$y_i$|4|4|4|4|4|4|3|...|
|$t_{q(i)}$|4|4|4|3|3|2|10|...|

Мы видим, что $y_{10}$ равен $y_{j=9..5}$. Но 0.5 можно добавлять благодаря взаимодействию с $j=7..5$ из-за того, что $t_{q(10)}=t_{q(i=9,8)}$. Находить эту границу с 7 по 5 элемент можно обычным линейным поиском, но это тоже долго, поэтому будем искать бинпоиском.
<br>
Сначала найдём границу $l_y:\ y_{l_y}<y_i,\ y_{l_y+1}=y_i$. В данном примере $l_y=4$.
<br>
Потом опять бинпоиском в диапазоне от $l_y+1$ до $i$ найдём такой $l_t:\ t_{q(l_t)}<t_{q(i)},\ t_{q(l_t+1)}=t_{q(i)}$. В данном примере $l_t=4$.
<br>
Получим интервал длиной $l_t-l_y$, именно он должен идти с коэффициентом 0.5. Кстати, так как мы всегда добавляем $bigger_{q(i)}$, на самом деле этот интервал уже посчитан с коэффициентом 1, поэтому надо будет от числителя отнимать $0.5*(l_t-l_y)$

**Замечание.** Очень важно искать $l_t$ в интервале $[l_y+1, i]$, так как в нём значения $t_{q(j)}$ отсортированы, а в общем нет. Если как в обычном бинпоиске брать $l=-1,r=N-1$, то мы будем искать в несортированном списке.

# Код

In [41]:
import math

class T:
    def __init__(self, val, ind):
        self.val = val
        self.ind = ind

class Y:
    def __init__(self, val, t):
        self.val = val
        self.t = t

class SegmentTree:
    def __init__(self, a):
        self.buildTree(a)
        
    def buildTree(self, a):
        self.t = [0] * 2**math.ceil(math.log2(len(a))) + a + [0] * (2**(math.ceil(math.log2(len(a)))) - len(a))
        self.zero_ind = 2**math.ceil(math.log2(len(a)))

    def __getitem__(self, ind):
        ind += self.zero_ind
        val = 0
        while ind > 0:
            val += self.t[ind]
            ind //= 2
        return val

    def update(self, l, r, val):
        l += self.zero_ind
        r += self.zero_ind
        while l < r:
            if l % 2 == 1:
                self.t[l] += val
                l = l // 2 + 1
            else:
                l //= 2
            if r % 2 == 0:
                self.t[r] += val
                r = r // 2 - 1
            else:
                r //= 2
        if l == r:
            self.t[l] += val

def binSearch(a, r):
    val = a[r].val
    ly, ry = -1, r
    while ry - ly > 1:
        m = (ly + ry) // 2
        if a[m].val < val:
            ly = m
        else:
            ry = m
    lt, rt = ly + 1, r
    t = a[r].t.val
    while rt - lt > 1:
        m = (lt + rt) // 2
        if a[m].t.val < t:
            lt = m
        else:
            rt = m
    if a[lt].t.val == t:
        lt = ly
    return max(0, lt - ly)

n = int(input())
t = []
y = []
for i in range(n):
    t_i, y_i = [float(x) for x in input().split()]
    t.append(T(t_i, i))
    y.append(Y(y_i, t[-1]))
t.sort(key=lambda x: x.val)
for i in range(n):
    t[i].ind = i
bigger = [0] # bigger[i] = number of t[j] smaller then t[i]
next_bigger = [n] * n # next_bigger[i] = min(j) where t[j] > t[i]
for i in range(1, n):
    bigger.append(i if t[i].val > t[i - 1].val else bigger[i - 1])
for i in range(n - 2, -1, -1):
    next_bigger[i] = i + 1 if t[i].val < t[i + 1].val else next_bigger[i + 1]
denominator = sum(bigger)
numerator = 0
bigger = SegmentTree(bigger)
y.sort(key=lambda x: (x.val, x.t.val))
for i in range(n - 1, -1, -1):
    numerator += bigger[y[i].t.ind] - binSearch(y, i) * 0.5
    bigger.update(next_bigger[y[i].t.ind], n - 1, -1)
print(numerator / denominator)

 10
 0 4
 3 0
 1 2
 2 4
 1 0
 2 1
 4 1
 2 1
 4 4
 0 0


0.5384615384615384
